In [3]:
!pip install -qU langchain_community

In [4]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 600, chunk_overlap = 0)

# 텍스트 파일을 load -> List[Document] 형태로 변환
loader1 = TextLoader("/content/nlp-keywords.txt")
loader2 = TextLoader("/content/finance-keywords.txt")

# 문서 분할
split_doc1 = loader1.load_and_split(text_splitter)
split_doc2 = loader2.load_and_split(text_splitter)

# 문서 개수 확인
len(split_doc1), len(split_doc2)

(11, 6)

# VectorStore 생성

In [5]:
!pip install -qU faiss-cpu langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.4 MB/s eta 0:00:00


In [6]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_openai import OpenAIEmbeddings

# 임베딩
embeddings = OpenAIEmbeddings()

# 임베딩 차원 크기 계산
dimension_size = len(embeddings.embed_query("Hello World!"))
print(dimension_size)

1536


In [17]:
# 임베딩 모델을 강제로 설정하는 방법

embeddings = OpenAIEmbeddings(model = "text-embedding-3-small")
#embeddings = OpenAIEmbeddings(model = "text-embeddings-3-large")

In [18]:
db = FAISS(
    embedding_function = embeddings,
    index = faiss.IndexFlatL2(dimension_size),
    docstore = InMemoryDocstore(), # 메모리 상주 휘발
    index_to_docstore_id = {},
)

In [19]:
db = FAISS.from_documents(split_doc1, embeddings)

In [20]:
# 문서 저장소 ID 확인
db.index_to_docstore_id

{0: '452e0476-febd-4d6f-b939-db929e5a0e7e',
 1: 'c5dfb58d-ecf4-4ec1-a906-77221a5871f1',
 2: 'f95ccd9c-13b8-4b41-a872-e4e37833fefe',
 3: '9e99a6bd-81a4-49d8-a5cd-b369c68cb8bc',
 4: '1a9fe5d1-7bfd-4c62-89e5-6cb26b54e50c',
 5: '3f8875ed-7e4c-4c09-b385-f9b61919dec0',
 6: '51e52998-ca2f-4b75-bac8-2cdaacbc2e5e',
 7: '7fab075b-d287-44b0-82c1-31c444627720',
 8: '14497bed-6230-4b26-9d77-96c664a6f71b',
 9: '87165231-af62-4b50-97c8-3f99c42ca814',
 10: 'f4cd4b29-22f3-49ea-9a5f-50ecaef11e8c'}

In [23]:
db.docstore._dict

{'452e0476-febd-4d6f-b939-db929e5a0e7e': Document(id='452e0476-febd-4d6f-b939-db929e5a0e7e', metadata={'source': '/content/nlp-keywords.txt'}, page_content='Semantic Search\n\n정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.\n예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.\n연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝\n\nEmbedding\n\n정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.\n예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.\n연관키워드: 자연어 처리, 벡터화, 딥러닝\n\nToken\n\n정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.\n예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다.\n연관키워드: 토큰화, 자연어 처리, 구문 분석\n\nTokenizer'),
 'c5dfb58d-ecf4-4ec1-a906-77221a5871f1': Document(id='c5dfb58d-ecf4-4ec1-a906-77221a5871f1', metadata={'source': '/content/nlp-keywords.txt'}, page_content='정의: 토크나이저는 텍스트 데이터를 토큰으로 분할하는 도구입니다. 이는 자연어 처리에서 데이터를 전처리하는 데 사용됩니다.\n예시: "I love programming."이라는 문장을 ["I", "love", "programming", "."]으로 분할합니

# from_text

In [25]:
db2 = FAISS.from_texts(
    ["1번 문서입니다.", "2번 문서입니다."],
    embedding = OpenAIEmbeddings(),
    metadatas = [{"source": "텍스트 문서"}, {"source": "텍스트 문서"}],
    ids = ["doc1", "doc2"],
)

In [26]:
db2.docstore._dict

{'doc1': Document(id='doc1', metadata={'source': '텍스트 문서'}, page_content='1번 문서입니다.'),
 'doc2': Document(id='doc2', metadata={'source': '텍스트 문서'}, page_content='2번 문서입니다.')}

# Similarity Search

**매개변수**

- `query (str)`: 유사한 문서를 찾기 위한 검색 쿼리 텍스트
-` k (int)`: 반환할 문서 수. 기본값은 4
- `filter` (Optional[Union[Callable, Dict[str, Any]]]): 메타데이터 필터링 함수 또는 딕셔너리. 기본값은 None
- `fetch_k (int)`: 필터링 전에 가져올 문서 수. 기본값은 20
- `**kwargs`: 추가 키워드 인자

In [28]:
# 유사도 검색
db.similarity_search("TF IDF 에 대하여 알려줘", k=2)

[Document(id='51e52998-ca2f-4b75-bac8-2cdaacbc2e5e', metadata={'source': '/content/nlp-keywords.txt'}, page_content='정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을 가집니다.\n연관키워드: 자연어 처리, 정보 검색, 데이터 마이닝\n\nDeep Learning\n\n정의: 딥러닝은 인공신경망을 이용하여 복잡한 문제를 해결하는 머신러닝의 한 분야입니다. 이는 데이터에서 고수준의 표현을 학습하는 데 중점을 둡니다.\n예시: 이미지 인식, 음성 인식, 자연어 처리 등에서 딥러닝 모델이 활용됩니다.\n연관키워드: 인공신경망, 머신러닝, 데이터 분석\n\nSchema\n\n정의: 스키마는 데이터베이스나 파일의 구조를 정의하는 것으로, 데이터가 어떻게 저장되고 조직되는지에 대한 청사진을 제공합니다.\n예시: 관계형 데이터베이스의 테이블 스키마는 열 이름, 데이터 타입, 키 제약 조건 등을 정의합니다.\n연관키워드: 데이터베이스, 데이터 모델링, 데이터 관리\n\nDataFrame'),
 Document(id='3f8875ed-7e4c-4c09-b385-f9b61919dec0', metadata={'source': '/content/nlp-keywords.txt'}, page_content='정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이나 스키마에 따라 조직된 데이터입니다. 

In [29]:
# filter 사용
db.similarity_search(
    "TF IDF 에 대하여 알려줘", filter = {"source": "/content/nlp-keywords.txt"}, k=2
)

[Document(id='51e52998-ca2f-4b75-bac8-2cdaacbc2e5e', metadata={'source': '/content/nlp-keywords.txt'}, page_content='정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을 가집니다.\n연관키워드: 자연어 처리, 정보 검색, 데이터 마이닝\n\nDeep Learning\n\n정의: 딥러닝은 인공신경망을 이용하여 복잡한 문제를 해결하는 머신러닝의 한 분야입니다. 이는 데이터에서 고수준의 표현을 학습하는 데 중점을 둡니다.\n예시: 이미지 인식, 음성 인식, 자연어 처리 등에서 딥러닝 모델이 활용됩니다.\n연관키워드: 인공신경망, 머신러닝, 데이터 분석\n\nSchema\n\n정의: 스키마는 데이터베이스나 파일의 구조를 정의하는 것으로, 데이터가 어떻게 저장되고 조직되는지에 대한 청사진을 제공합니다.\n예시: 관계형 데이터베이스의 테이블 스키마는 열 이름, 데이터 타입, 키 제약 조건 등을 정의합니다.\n연관키워드: 데이터베이스, 데이터 모델링, 데이터 관리\n\nDataFrame'),
 Document(id='3f8875ed-7e4c-4c09-b385-f9b61919dec0', metadata={'source': '/content/nlp-keywords.txt'}, page_content='정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이나 스키마에 따라 조직된 데이터입니다. 

# Document 추가

In [31]:
from langchain_core.documents import Document

db.add_documents(
    [
        Document(
            page_content = "안녕하세요! 이번엔 도큐먼트를 새로 추가해 볼께요.",
            metadata = {"source": "mydata.txt"},
        )
    ],
    ids = ["new_doc1"],
)

['new_doc1']

In [32]:
# 추가된 데이터 확인
db.similarity_search("안녕하세요", k = 2)

[Document(id='new_doc1', metadata={'source': 'mydata.txt'}, page_content='안녕하세요! 이번엔 도큐먼트를 새로 추가해 볼께요.'),
 Document(id='f95ccd9c-13b8-4b41-a872-e4e37833fefe', metadata={'source': '/content/nlp-keywords.txt'}, page_content='정의: CSV(Comma-Separated Values)는 데이터를 저장하는 파일 형식으로, 각 데이터 값은 쉼표로 구분됩니다. 표 형태의 데이터를 간단하게 저장하고 교환할 때 사용됩니다.\n예시: 이름, 나이, 직업이라는 헤더를 가진 CSV 파일에는 홍길동, 30, 개발자와 같은 데이터가 포함될 수 있습니다.\n연관키워드: 데이터 형식, 파일 처리, 데이터 교환\n\nJSON\n\n정의: JSON(JavaScript Object Notation)은 경량의 데이터 교환 형식으로, 사람과 기계 모두에게 읽기 쉬운 텍스트를 사용하여 데이터 객체를 표현합니다.\n예시: {"이름": "홍길동", "나이": 30, "직업": "개발자"}는 JSON 형식의 데이터입니다.\n연관키워드: 데이터 교환, 웹 개발, API\n\nTransformer\n\n정의: 트랜스포머는 자연어 처리에서 사용되는 딥러닝 모델의 한 유형으로, 주로 번역, 요약, 텍스트 생성 등에 사용됩니다. 이는 Attention 메커니즘을 기반으로 합니다.\n예시: 구글 번역기는 트랜스포머 모델을 사용하여 다양한 언어 간의 번역을 수행합니다.\n연관키워드: 딥러닝, 자연어 처리, Attention\n\nHuggingFace')]

# Text 추가

In [33]:
db.add_texts(
    ["이번엔 텍스트 데이터를 추가합니다.", "추가한 2번째 텍스트 데이터입니다."],
    metadatas = [{"source": "mydata.txt"}, {"source": "mydata.txt"}],
    ids = ["new_doc4", "new_doc5"],
)

['new_doc4', 'new_doc5']

In [34]:
db.index_to_docstore_id

{0: '452e0476-febd-4d6f-b939-db929e5a0e7e',
 1: 'c5dfb58d-ecf4-4ec1-a906-77221a5871f1',
 2: 'f95ccd9c-13b8-4b41-a872-e4e37833fefe',
 3: '9e99a6bd-81a4-49d8-a5cd-b369c68cb8bc',
 4: '1a9fe5d1-7bfd-4c62-89e5-6cb26b54e50c',
 5: '3f8875ed-7e4c-4c09-b385-f9b61919dec0',
 6: '51e52998-ca2f-4b75-bac8-2cdaacbc2e5e',
 7: '7fab075b-d287-44b0-82c1-31c444627720',
 8: '14497bed-6230-4b26-9d77-96c664a6f71b',
 9: '87165231-af62-4b50-97c8-3f99c42ca814',
 10: 'f4cd4b29-22f3-49ea-9a5f-50ecaef11e8c',
 11: 'new_doc1',
 12: 'new_doc4',
 13: 'new_doc5'}

# 데이터 삭제

In [36]:
ids = db.add_texts(
    ["삭제용 데이터를 추가합니다.", "2번째 삭제용 데이터입니다."],
    metadatas = [{"source": "mydata.txt"}, {"source": "mydata.txt"}],
    ids = ["delete_doc1", "delete_doc2"],
)

In [37]:
# 삭제 전
db.index_to_docstore_id

{0: '452e0476-febd-4d6f-b939-db929e5a0e7e',
 1: 'c5dfb58d-ecf4-4ec1-a906-77221a5871f1',
 2: 'f95ccd9c-13b8-4b41-a872-e4e37833fefe',
 3: '9e99a6bd-81a4-49d8-a5cd-b369c68cb8bc',
 4: '1a9fe5d1-7bfd-4c62-89e5-6cb26b54e50c',
 5: '3f8875ed-7e4c-4c09-b385-f9b61919dec0',
 6: '51e52998-ca2f-4b75-bac8-2cdaacbc2e5e',
 7: '7fab075b-d287-44b0-82c1-31c444627720',
 8: '14497bed-6230-4b26-9d77-96c664a6f71b',
 9: '87165231-af62-4b50-97c8-3f99c42ca814',
 10: 'f4cd4b29-22f3-49ea-9a5f-50ecaef11e8c',
 11: 'new_doc1',
 12: 'new_doc4',
 13: 'new_doc5',
 14: 'delete_doc1',
 15: 'delete_doc2'}

In [38]:
print(ids)

['delete_doc1', 'delete_doc2']


In [39]:
db.delete(ids)

True

In [40]:
# 삭제 후
db.index_to_docstore_id

{0: '452e0476-febd-4d6f-b939-db929e5a0e7e',
 1: 'c5dfb58d-ecf4-4ec1-a906-77221a5871f1',
 2: 'f95ccd9c-13b8-4b41-a872-e4e37833fefe',
 3: '9e99a6bd-81a4-49d8-a5cd-b369c68cb8bc',
 4: '1a9fe5d1-7bfd-4c62-89e5-6cb26b54e50c',
 5: '3f8875ed-7e4c-4c09-b385-f9b61919dec0',
 6: '51e52998-ca2f-4b75-bac8-2cdaacbc2e5e',
 7: '7fab075b-d287-44b0-82c1-31c444627720',
 8: '14497bed-6230-4b26-9d77-96c664a6f71b',
 9: '87165231-af62-4b50-97c8-3f99c42ca814',
 10: 'f4cd4b29-22f3-49ea-9a5f-50ecaef11e8c',
 11: 'new_doc1',
 12: 'new_doc4',
 13: 'new_doc5'}

# Retriever(as_Retriever)

In [41]:
db = FAISS.from_documents(
    documents = split_doc1 + split_doc2, embedding = OpenAIEmbeddings()
)

In [43]:
retriever = db.as_retriever()
retriever.invoke("안녕하세요.")

[Document(id='7db3a208-3336-44fd-8c80-0fe39a3f2ce4', metadata={'source': '/content/nlp-keywords.txt'}, page_content='정의: CSV(Comma-Separated Values)는 데이터를 저장하는 파일 형식으로, 각 데이터 값은 쉼표로 구분됩니다. 표 형태의 데이터를 간단하게 저장하고 교환할 때 사용됩니다.\n예시: 이름, 나이, 직업이라는 헤더를 가진 CSV 파일에는 홍길동, 30, 개발자와 같은 데이터가 포함될 수 있습니다.\n연관키워드: 데이터 형식, 파일 처리, 데이터 교환\n\nJSON\n\n정의: JSON(JavaScript Object Notation)은 경량의 데이터 교환 형식으로, 사람과 기계 모두에게 읽기 쉬운 텍스트를 사용하여 데이터 객체를 표현합니다.\n예시: {"이름": "홍길동", "나이": 30, "직업": "개발자"}는 JSON 형식의 데이터입니다.\n연관키워드: 데이터 교환, 웹 개발, API\n\nTransformer\n\n정의: 트랜스포머는 자연어 처리에서 사용되는 딥러닝 모델의 한 유형으로, 주로 번역, 요약, 텍스트 생성 등에 사용됩니다. 이는 Attention 메커니즘을 기반으로 합니다.\n예시: 구글 번역기는 트랜스포머 모델을 사용하여 다양한 언어 간의 번역을 수행합니다.\n연관키워드: 딥러닝, 자연어 처리, Attention\n\nHuggingFace'),
 Document(id='e45d5b99-ce1f-4a2a-98a4-40dd6b445033', metadata={'source': '/content/nlp-keywords.txt'}, page_content='정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니다.\n예시: HuggingFace의 Transformers 라이브러리를 사용하여 감정 분석,

In [44]:
retriever = db.as_retriever(
    search_type = "mmr", search_kwargs = {"k": 6, "lambda_mult": 0.25, "fetch_k": 10}
)

retriever.invoke("Word2Vec에 대해 알려줘.")

[Document(id='bbb1d6b8-ac0c-42cc-a7a9-554aabdb5270', metadata={'source': '/content/nlp-keywords.txt'}, page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)\n\n정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이해 및 생성 작업에 사용됩니다.\n예시: OpenAI의 GPT 시리즈는 대표적인 대규모 언어 모델입니다.\n연관키워드: 자연어 처리, 딥러닝, 텍스트 생성\n\nFAISS (Facebook AI Similarity Search)\n\n정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source'),
 Document(id='e81cbbc2-01e1-4bfe-9b17-b7ed5819f58f', metadata={'source': '/content/nlp-keywords.txt'}, page_content='GPT (Generative Pretrained Transformer)\n\n정의: GPT는 대규모의 데이터셋으로 사전 훈련된 생성적 언어 모델로, 다양한 텍스트 기반 작업에 활용됩니다. 이는 입력된 텍스트에 기반하여 자연스러운 언어를 생성할 수 있습니다.\n예시: 사용자가 제공한 질문에 대해 자

In [47]:
# 임계값 기반 검색 수행
retriever = db.as_retriever(
    search_type = "similarity_score_threshold", search_kwargs = {"score_threshold": 0.8}
)

retriever.invoke("Word2Vec에 대해 알려줘.")

[Document(id='bbb1d6b8-ac0c-42cc-a7a9-554aabdb5270', metadata={'source': '/content/nlp-keywords.txt'}, page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)\n\n정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이해 및 생성 작업에 사용됩니다.\n예시: OpenAI의 GPT 시리즈는 대표적인 대규모 언어 모델입니다.\n연관키워드: 자연어 처리, 딥러닝, 텍스트 생성\n\nFAISS (Facebook AI Similarity Search)\n\n정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source')]

In [48]:
# 메타데이터 필터 적용
retriever = db.as_retriever(
    search_kwargs={"filter": {"source": "/content/finance-keywords.txt"}, "k": 2}
)
retriever.invoke("ESG 에 대하여 알려줘")

[Document(id='4ab7c902-02cc-41c6-9778-f763d3748cb1', metadata={'source': '/content/finance-keywords.txt'}, page_content='정의: ESG는 기업의 환경, 사회, 지배구조 측면을 고려하는 투자 접근 방식입니다.\n예시: S&P 500 ESG 지수는 우수한 ESG 성과를 보이는 기업들로 구성된 지수입니다.\n연관키워드: 지속가능 투자, 기업의 사회적 책임, 윤리 경영\n\nStock Buyback\n\n정의: 자사주 매입은 기업이 자사의 주식을 시장에서 다시 사들이는 것을 말합니다.\n예시: 애플은 S&P 500 기업 중 가장 큰 규모의 자사주 매입 프로그램을 운영하고 있습니다.\n연관키워드: 주주 가치, 자본 관리, 주가 부양\n\nCyclical Stocks\n\n정의: 경기순환주는 경제 상황에 따라 실적이 크게 변동하는 기업의 주식을 말합니다.\n예시: 포드, 제너럴 모터스와 같은 자동차 기업들은 S&P 500에 포함된 대표적인 경기순환주입니다.\n연관키워드: 경제 사이클, 섹터 분석, 투자 타이밍\n\nDefensive Stocks\n\n정의: 방어주는 경기 변동에 상관없이 안정적인 실적을 보이는 기업의 주식을 의미합니다.\n예시: 프록터앤갬블, 존슨앤존슨과 같은 생활필수품 기업들은 S&P 500 내 대표적인 방어주로 꼽힙니다.\n연관키워드: 안정적 수익, 저변동성, 리스크 관리'),
 Document(id='196d69eb-69d4-444e-aa10-6c9babbeee61', metadata={'source': '/content/finance-keywords.txt'}, page_content='정의: 주식 리서치는 기업의 재무 상태, 사업 모델, 경쟁력 등을 분석하여 투자 의사 결정을 돕는 활동입니다.\n예시: 골드만삭스의 애널리스트들이 S&P 500 기업들에 대한 분기별 실적 전망을 발표했습니다.\n연관키워드: 투자 분석, 기업 가치평가, 시장 전망\n\nCo